In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [7]:
import sklearn.metrics as metrics
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import NearestNeighbors


In [ ]:
df = pd.read_csv("./")

In [ ]:
def preprocess(df):



    return X, x_train, x_test, y_train, y_test

In [ ]:
def compute_scores(y_test, y_pred):
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse) 
    r2 = metrics.r2_score(y_test, y_pred)
    print("MAE : ",mae)
    print("MSE : ",mse)
    print("RMSE : ",rmse)
    print("R2 : ",r2)

In [ ]:
X, x_train, x_test, y_train, y_test, x_scaled = preprocess(df)

In [ ]:
correlation_matrix = pd.DataFrame(X).corr()
plt.matshow(correlation_matrix)
plt.show()

#### Linear Regression

In [ ]:
model_lin = LinearRegression(random_state = 13)
model_lin.fit(x_train, y_train)
y_pred_linear= model_lin.predict(x_test)
compute_scores(y_test, y_pred_linear)

#### Decision Tree

In [ ]:
dt_regression = DecisionTreeRegressor(random_state = 13)
dt_regression.fit(x_train, y_train)
y_pred_dt = dt_regression.predict(x_test)
compute_scores(y_test, y_pred_dt)

#### KNN

In [ ]:
kNN = NearestNeighbors(n_neighbors=1, random_state = 13) #tweak number of neighbours
kNN.fit(x_train, y_train)
y_pred_knn = kNN.predict(x_test)
compute_scores(y_test, y_pred_knn)

#### Shrinkage - Ridge

In [5]:
ridge_regression = Ridge(alpha=1.0, solver = '', random_state = 13) #if we use alpha = 0, it becomes simple LR problem, solver = ‘auto’, ‘svd’, ‘cholesky’, ‘lsqr’, ‘sparse_cg’, ‘sag’, ‘saga’, ‘lbfgs’
ridge_regression.fit(x_train, y_train)
y_pred_ridge = ridge_regression.predict(x_test)
compute_scores(y_test, y_pred_ridge)

NameError: name 'X' is not defined

#### Shrinkage - Lasso

In [ ]:
lasso_regression = linear_model.Lasso(alpha=0.1, random_state = 13)
lasso_regression.fit(x_train, y_train)
y_pred_lasso = lasso_regression.predict(x_test)
compute_scores(y_test, y_pred_lasso)